In [2]:

import os

from simulation import Simulation

file_path = os.path.join(os.path.dirname(os.getcwd()), "src", "simulation_config.json")

sim = Simulation.from_json(file_path, enable_viz=False, enable_log=True)

In [ ]:
import casadi as ca

def split_jacobian_x_u(f):
    """
    Given f(x, u), return two functions:
      - f_jac_x(x, u): Jacobian wrt x
      - f_jac_u(x, u): Jacobian wrt u
    """
    assert f.n_in() == 2, "f must have exactly two inputs (x, u)"

    jac_f = f.jacobian()

    def _zero_seeds(x_val, u_val):
        outs = f(x_val, u_val)
        if f.n_out() == 1:
            outs = (outs,)
        return [ca.DM.zeros(o.size()) for o in outs]

    def f_jac_x(x_val, u_val):
        seeds = _zero_seeds(x_val, u_val)
        jac_blocks = jac_f(x_val, u_val, *seeds)

        # Blocks are ordered: (out1_x, out1_u, out2_x, out2_u, ...)
        return tuple(jac_blocks[::2])

    def f_jac_u(x_val, u_val):
        seeds = _zero_seeds(x_val, u_val)
        jac_blocks = jac_f(x_val, u_val, *seeds)

        return tuple(jac_blocks[1::2])

    return f_jac_x, f_jac_u

x = ca.SX.sym("x", 2)
u = ca.SX.sym("u", 1)

y1 = ca.vertcat(x[0] + u, x[1] * u)
y2 = x[0]**2 + u

f = ca.Function("f", [x, u], [y1, y2])

f_jac_x, f_jac_u = split_jacobian_x_u(f)

x_val = [1, 2]
u_val = [3]

Jx = f_jac_x(x_val, u_val)
Ju = f_jac_u(x_val, u_val)


In [ ]:
from io import StringIO
import sys
from IPython.display import Latex
import casadi as ca
import re
import numpy as np


def latex_print(expr, sub=False):
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    expr.print_sparse()
    sys.stdout = old_stdout
    x = mystdout.getvalue().split('\n')

    shape_split = x[0].split(':')[1].split(',')[0].strip().split('-')
    m = int(shape_split[0])
    n = int(shape_split[2])
    e = {}
    b = {
        '*': r' \cdot ',
        'cos': r'\cos',
        'sin': r'\sin'
    }
    for l in ['alpha', 'beta', 'gamma', 'Gamma',
            'delta', 'Delta', 'epsilon', 'zeta', 'eta', 'theta',
            'Theta', 'iota', 'kappa', 'lambda', 'Lambda', 'mu',
            'nu', 'xi', 'pi', 'rho', 'sigma', 'tau', 'upsilon',
            'phi', 'Phi', 'chi', 'psi', 'Psi', 'omega', 'Omega']:
        b[l] = '\\' + l
    s = r'\b(' + '|'.join(sorted(b.keys(), key=len, reverse=True)) + r')\b'
    # fix for escaped keys
    s = s.replace('*', '\\*')
    b_pattern = re.compile(s)
    t = {}
    for i in range(1, len(x)):
        line = x[i].strip()
        if line[0] == "@":
            split = line.split("=")
            term = split[1].replace(',', '')
            term = b_pattern.sub(lambda x: b[x.group()], term)
            e[split[0]] = term
        elif line[0] == "(":
            split = line.split("->")
            indices = split[0].strip().replace('(', '').replace(')', '').split(',')
            i = int(indices[0].strip())
            j = int(indices[1].strip())
            t[(i, j)] = str(split[1])
    s = r"\begin{bmatrix}"
    for i in range(m):
        row = []
        for j in range(n):
            term = "0";
            if (i, j) in t.keys():
                term = t[(i, j)]
            term = b_pattern.sub(lambda x: b[x.group()], term)
            if sub:
                for k in sorted(e.keys(), key=len, reverse=True):
                    term = term.replace(k, e[k])
            row += [r"{:s}".format(term)]
        s += "&".join(row) + r"\\"
    s += r"\end{bmatrix}"
    if not sub:
        s += r"\begin{align}"
        for k in e.keys():
            s += r"{:s}: && {:s} \\".format(k, e[k])
        s += r"\end{align}"
    return s

def latex_display(x, sub=True):
    return Latex(latex_print(x, sub))


# symbolic dynamics model for estimation and control
q_BI = ca.SX.sym('q', 4)
omega = ca.SX.sym(r'\omega', 3)
omega_w = ca.SX.sym(r"\Omega", 3)
x = ca.vertcat(q_BI, omega, omega_w)
u = ca.SX.sym('u', 6)
u_rw = u[:3]
u_mag = u[3:]

B_B = ca.SX.sym('B', 3) # where B is the measured magnetic field from the                   
A_hat = np.eye(3)
J_hat = np.eye(3)
J_w = np.ones(3)
K_t_dash = 1
K_e_rw = 1
K_mag = 1

# actuators
h_int = A_hat @ (J_w * (omega_w + A_hat @ omega)) 
tau_rw = K_t_dash * (A_hat @ (u_rw - K_e_rw * omega_w)) 

# r_norm = ca.norm_2(r_ECI) # instead of dipole model the measurements are used
# r_hat = r_ECI / r_norm
# B_I = (3 * r_hat * (ca.dot(M_dipole, r_hat)) - M_dipole) / r_norm**3
tau_mag = ca.cross(K_mag * u_mag, B_B) 

# attitude kinematics
q_dot = ca.SX.sym(r"\dot{q}", 4)
q_dot[:3] = 0.5 * (omega * q_BI[3] + ca.cross(omega, q_BI[:3]))
q_dot[3] = -0.5 * ca.dot(omega, q_BI[:3])

# attitude dynamics
cross_term = ca.cross(omega, J_hat @ omega + h_int)
total_torque = tau_mag - tau_rw - cross_term
omega_dot = ca.solve(J_hat, total_torque)
# omega_w_dot = u_rw / J_w - A_hat @ omega_dot


dx = ca.vertcat(q_dot, omega_dot)
f = ca.Function("f", [x, u, B_B], [dx], ["x", "u", "B"], ["dx"])

latex_display(dx)

In [ ]:
q = ca.SX.sym("q", 3)

cross = ca.vertcat(ca.horzcat(0, -q[2], q[1]), ca.horzcat(q[2], 0, -q[0]), ca.horzcat(-q[1], q[0], 0))
R = (1 - 2*ca.dot(q, q))**2 * ca.DM_eye(3) + 2* q @ ca.transpose(q) - 2* ca.sqrt(1-ca.dot(q, q)) * cross


# R_1 = ca.jacobian(R, q[0])
# R_2 = ca.jacobian(R, q[1])
# R_3 = ca.jacobian(R, q[2])

print(cross)
latex_display(cross)

In [ ]:
import sympy as sp

# 1. Define the symbolic variables
# Equivalent to: q = ca.SX.sym("q", 3)
q_syms = sp.symbols('q0:3')  # Creates symbols q0, q1, q2
q = sp.Matrix(q_syms)        # Creates a 3x1 column vector

# 2. Construct the skew-symmetric 'cross' matrix
# Equivalent to the vertcat/horzcat lines
cross = sp.Matrix([
    [0,       -q[2],   q[1]],
    [q[2],    0,      -q[0]],
    [-q[1],   q[0],    0]
])

# 3. Define intermediate terms for R
# Equivalent to: ca.dot(q, q)
q_dot_q = q.dot(q)

# Equivalent to: q @ ca.transpose(q)
# In SymPy, * performs matrix multiplication
q_outer = q * q.T

# 4. Calculate R
# Equivalent to: ca.DM_eye(3) -> sp.eye(3)
# Equivalent to: ca.sqrt(...) -> sp.sqrt(...)
term1 = (1 - 2 * q_dot_q)**2 * sp.eye(3)
term2 = 2 * q_outer
term3 = 2 * sp.sqrt(1 - q_dot_q) * cross

R = term1 + term2 - term3

# 5. Calculate the derivatives
# Equivalent to: ca.jacobian(R, q[i])
# Note: SymPy's .diff() returns a 3x3 Matrix of derivatives.
# If you need a flattened 9x1 vector like CasADi might return, 
# you can use R.reshape(9, 1).diff(q[0])
R_1 = R.diff(q[0])
R_2 = R.diff(q[1])
R_3 = R.diff(q[2])

# Optional: Print one of the results to verify
# sp.pprint(R_1)
R_3

In [ ]:
w_syms = sp.symbols("w0:3")
w = sp.Matrix(w_syms)


Rw_jac = sp.Matrix.hstack(R_1 * w, R_2 * w, R_3 * w)

Rw_jac.subs([(q_syms[0], 0), (q_syms[1], 0), (q_syms[2], 0)])

In [ ]:
J_syms = sp.symbols(r"J(:3\,:3)")
J = sp.Matrix(np.array(J_syms).reshape(3, 3))

Jw = J * w

Jw_cross = sp.Matrix([
    [0,       -Jw[2],   Jw[1]],
    [Jw[2],    0,      -Jw[0]],
    [-Jw[1],   Jw[0],    0]
])
w_cross = sp.Matrix([
    [0,       -w[2],   w[1]],
    [w[2],    0,      -w[0]],
    [-w[1],   w[0],    0]
])

A_21 = 2 * J**-1 * (Jw_cross - w_cross * J) * w_cross

A_21.subs([(w[0], 0), [w[2], 0]])

In [ ]:
R_2.subs([(q_syms[0], 0), (q_syms[1], 0), (q_syms[2], 0)]) * w

In [ ]:
import datetime
import json
from dynamics import SGP4
from kinematics import euler_ocr_to_sbc
from satellite import Spacecraft
from simulation import Simulation

def string_to_timedelta(total_time: str) -> datetime.timedelta:
    match tuple(map(float, total_time.split(':'))):
        case (hours, minutes, seconds):
            return datetime.timedelta(hours=hours, minutes=minutes, seconds=seconds)
        case (minutes, seconds):
            return datetime.timedelta(minutes=minutes, seconds=seconds)
        case (seconds,):
            return datetime.timedelta(seconds=seconds)
    raise ValueError(f"String '{total_time}' not in format h:m:s")


file_path = ""
with open(file_path, "r") as f:
    data = json.load(f)

data_sim = data["Simulation"] 
t0 = datetime.datetime.fromisoformat(data_sim["Start"])
if t0.tzinfo is None:
    t0 = t0.replace(tzinfo=datetime.timezone.utc)

dt = string_to_timedelta(data_sim["Stepsize"])

dur = string_to_timedelta(data_sim["Duration"])

tf = t0 + dur



tle1 = data["InitialState"]["TLE"]["Line 1"]
tle2 = data["InitialState"]["TLE"]["Line 2"]
orbit_model = SGP4.from_tle(tle1, tle2)
r_ECI, v_ECI = orbit_model.propagate(t0)

roll = data_sim["InitialAttitude"]["Roll (deg)"]
pitch = data_sim["InitialAttitude"]["Pitch (deg)"]
yaw = data_sim["InitialAttitude"]["Yaw (deg)"]

R_BO = euler_ocr_to_sbc(roll, pitch, yaw)

ang_vel_B_BO = np.array(data_sim["InitialRates (wrt ORC in SBC)"])

orbit_ang_vel = np.linalg.norm(v_ECI)/np.linalg.norm(r_ECI)
init_ang_vel_B_BI = ang_vel_B_BO + R_BO.apply(np.array((0, -orbit_ang_vel, 0)))

Simulation(Spacecraft.from_dict(data["SpacecraftParams"]), r_ECI, v_ECI, R_BO, init_ang_vel_B_BI, dt, t0, tf, enable_viz, enable_log)

In [7]:
import numpy as np
import json 
a = np.eye(3)

jsstring = json.dumps({"arr": a.tolist(), "bla": 4})

np.array(json.loads(jsstring)["arr"])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])